In [ ]:
import re
import os
import time
import random
import requests
import numpy as np
import pandas as pd
from os import system
from math import floor
from copy import deepcopy
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
from selenium.webdriver.chrome.options import Options
import config
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 200)

In [ ]:
mobile_emulation = { "deviceName": "iPhone X" }

chrome_options = Options()

chrome_options.add_experimental_option("mobileEmulation", mobile_emulation)

driver = webdriver.Chrome(chrome_options = chrome_options)

In [ ]:
driver.get('https://stocktwits.com/')

In [ ]:
login = driver.find_element_by_xpath('//*[@id="app"]/div/div/div[2]/div/div/div/div/div[1]/div[1]/div[4]/div/span[2]')

In [ ]:
login.click()

In [ ]:
#username
username = driver.find_element_by_xpath('//*[@id="app"]/div/div/div[4]/div[2]/div/form/div[1]/div[1]/label/input')
username.click()
username.send_keys(config.username)


In [ ]:
#password
password = driver.find_element_by_xpath('//*[@id="app"]/div/div/div[4]/div[2]/div/form/div[1]/div[2]/label/input')
password.click()
password.send_keys(config.password)

In [ ]:
#sign in 
signin = driver.find_element_by_xpath('//*[@id="app"]/div/div/div[4]/div[2]/div/form/div[2]/div[1]/button')
signin.click()

In [ ]:
from datetime import date
today = date.today()


In [ ]:
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
tickers = ['aapl', 'fb', 'spy', 'cost', 't', 'cgc', 'amd', 'sbux', 'f', 'msft', 'ba', 'roku', 'baba', 'goog', 'amzn', 'nflx']
# tickers = ['f', 'msft', 'ba', 'roku', 'baba', 'goog', 'amzn', 'nflx']

stock_dic = {}
stock_list = []
key = ''
ts = TimeSeries(key)

datecurrent_date = today.strftime("%Y-%m-%d")

In [ ]:
for i in tickers:
    key = ''
    ts = TimeSeries(key)
    driver.get(f'https://stocktwits.com/symbol/{i}')
    sentiment = driver.find_element_by_xpath('//*[@id="app"]/div/div/div[2]/div[2]/div[2]/div[1]/div/main/section/div[1]/div/div/div/div/div[1]/div/div/div[2]/div/div[1]')
    sent = sentiment.text
    try:
        sent_color = driver.find_element_by_class_name('st_37VuZWc')
    except:
        sent_color = driver.find_element_by_class_name('st_2ViIaO9')
    
    if sent_color.value_of_css_property('color') == 'rgba(255, 62, 62, 1)':
        stock_dic['Direction'] = 'Negetive'
    else:
        stock_dic['Direction'] = 'Positve'
    stock_dic['Stock'] = i
    stock_dic['Date'] = datecurrent_date
    stock_dic[f'{i}_Sentiment'] = sent
    stock, meta = ts.get_daily(symbol=i)
    stock_dic[f'{i}_Open_Price'] = stock[f'{datecurrent_date}']['1. open']
    stock_dic[f'{i}_Close_Price'] = stock[f'{datecurrent_date}']['4. close']
    stock_dic[f'{i}_Volume'] = stock[f'{datecurrent_date}']['5. volume']
    stock_list.append(stock_dic)
    stock_dic = {}
    if i != tickers[-1]:
        time.sleep(14)

In [ ]:
stock_list